# Práctica 2: Árboles de Decisión

Inteligencia de Datos

Universidad Iberoamericana Ciudad de México

Creado por: Luis Norberto Zúñiga Morales, inspirado en el libro *Hands-On Machine Learning with Scikit-Learn, Keras & Tensorflow*.

Los árboles de decisión (AD) son modelos muy versátiles que permiten realizar tanto clasificación como regresión. Además, como veremos más adelante, son el bloque fundamental de modelos más complejos, como los Bosques Aleatorios.

En esta práctica vamos a explorar como entrenar, visualizar y hacer visualizaciones con AD en scikit-learn. Después, exploraremos como regularizar estos modelos.


---



## Entrenamiento y visualización de Árboles de Decisión

Para comprender mejor la idea que permite que los AD funciones, vamos a construir uno de estos modelos y dar un vistazo a las predicciones que realiza. Iniciemos entrenando un `DecisionTreeClassifier` con el conjunto de datos Iris:

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris = load_iris()
X = iris.data[:, 2:] # largo y ancho del petalo
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

**Actividad**: Revisen lo documentación de `DecisionTreeClassifier` y expliquen qué es lo que hacen los primeros cinco parámetros de la clase.

*Su respuesta en esta celda*

¿Cómo se ve el árbol entrenado? Por ver, nos referimos a las decisiones que toma en cada nodo.

In [ ]:
from graphviz import Source
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file="iris_tree.dot",
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

In [ ]:
Source.from_file("iris_tree.dot")

** Actividad**: Supongan que encuentran una flor Iris, a la cual le sacan sus medidas. ¿Cómo interpretan el árbol anterior para determinar la clase de la nueva flor?

*Su respuesta en esta celda*

## Predicciones con Árboles de Decisión

Scikit-learn puede usar distintos criterios para medir la calidad de un partición en un nodo. En clase vimos el criterio de entropía y de tarea se dejó investigar el criterio de Gini.

A continuación, vamos a graficar cada partición (o decisión) hecha en el árbol.

In [ ]:
import numpy as np

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [ ]:
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Largo del pétalo", fontsize=14)
        plt.ylabel("Ancho del pétalo", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

plt.figure(figsize=(8, 4))
plot_decision_boundary(tree_clf, X, y)
plt.plot([2.45, 2.45], [0, 3], "k-", linewidth=2)
plt.plot([2.45, 7.5], [1.75, 1.75], "k--", linewidth=2)
plt.plot([4.95, 4.95], [0, 1.75], "k:", linewidth=2)
plt.plot([4.85, 4.85], [1.75, 3], "k:", linewidth=2)
plt.text(1.40, 1.0, "Depth=0", fontsize=13)
plt.text(3.2, 1.80, "Depth=1", fontsize=13)
plt.text(4.05, 0.5, "(Depth=2)", fontsize=11)

plt.show()

La linea sólida representa el límite de decisión que genera el primer nodo (longitud del pétalo = 2.45 cm). Noten que esta regla separa perfectamente los puntos verdes de los azules, por lo que no se deben crear más particiones, lo cual no sucede en el lado derecho. Por lo tanto, se crea una nueva partición (ancho del pétalo = 1.75 cm) representada por la línea punteada sólida. Dado que usamos el parámetro `max_depth` con un valor de 2, el algoritmo detiene su ejecución en ese punto. Si usaramos un valor de 3, se añadiría otra partición en la parte impura, que es el resultado que se observa con las líneas punteadas más delgadas.

Los árboles de decisión son intuitivos y las decisiones son relativamente fáciles de entender. Es decir, son modelos de fácil interpretación (o *cajas blancas*). En contraste, modelo más complejos como las redes neuronales no poseen estas carecterísticas, lo que los vuelven poco interpretables (*cajas negras*).

## Regularización e hiperparámetros

Los árboles de decisión hacen pocas suposiciones sobre los datos de entrenamiento. Si no se controlan, se pueden ajustar demasiado a los datos con los que se entrena, potencialmente ocasionando sobreajuste. Para evitar el sobreajuste se deben restringir algunos parámetros del modelo durante su entrenamiento, es decir, regularizarlo. En este caso, lo que se controla son algunos hiperparámetros del modelo, principalmente la profundida máxima del AD, controlada por el parámetro `max_depth`.

Otros parámetros que se deben controlar son:

- `min_samples_split`: el número mínimo de muestras que un nodo de contener antes de que se particione.
- `min_samples_leaf`: número mínimo de muestras que un nodo debe contener.
- `min_weight_fraction_leaf`: igual que el anterior, pero expresado como una fracción del total de instancias ponderadas.
- `max_leaf_nodes`: número máximo de nodos hoja.
- `max_features`: el número máximo de características que se evaluan para partir cada nodo.

Para regularizar, debemos incrementar todos los `min_` o reducir todos los `max_` y encontrar un balance.

En el siguiente código vamos a crear un conjunto de datos artificial con la función `make_moons` y entrenar dos modelos: el primero sin restricciones y el segundo controloando el parámetro `min_samples_leaf`. Después, vamos a visualizar como particiona el espacio de los datos para comparar su efecto.

In [ ]:
from sklearn.datasets import make_moons
Xm, ym = make_moons(n_samples=100, noise=0.25, random_state=53)

deep_tree_clf1 = DecisionTreeClassifier(random_state=42)
deep_tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4, random_state=42)
deep_tree_clf1.fit(Xm, ym)
deep_tree_clf2.fit(Xm, ym)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(deep_tree_clf1, Xm, ym, axes=[-1.5, 2.4, -1, 1.5], iris=False)
plt.title("Sin restricciones", fontsize=16)
plt.sca(axes[1])
plot_decision_boundary(deep_tree_clf2, Xm, ym, axes=[-1.5, 2.4, -1, 1.5], iris=False)
plt.title("min_samples_leaf = {}".format(deep_tree_clf2.min_samples_leaf), fontsize=14)
plt.ylabel("")

plt.show()

**Actividad**: ¿Qué pueden observar en la gráfica?

*Su respuesta en esta celda*

In [ ]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

## Árboles de Decisión para Regresión

Como se dejo en la tarea, los árboles de decisión también permiten resolver problemas de regresión. Vamos a implementar un AD usando la clase `DecisionTreeRegressor` en un conjunto de datos cuadrático con ruido. Empezamos creando los datos:

In [ ]:
np.random.seed(42)
m = 200
X = np.random.rand(m, 1)
y = 4 * (X - 0.5) ** 2
y = y + np.random.randn(m, 1) / 10

Es turno de crear el modelo de regresión usando AD:

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg.fit(X, y)

Una vez entrenado, vamos a observar las reglas que determinó para particionar los datos:

In [ ]:
export_graphviz(
        tree_reg,
        out_file="regression_tree.dot",
        feature_names=["x1"],
        rounded=True,
        filled=True
    )

Source.from_file( "regression_tree.dot")

El AD anterior ahora busca predecir un valor numérico en lugar de una clase. Por ejemplo, si quisieramos determinar el valor de un nuevo dato $x_1=0.6$, empezamos en el nodo de origen, nos vamos al nodo de la derecha, y de ahí al nodo de la izquierda, obteniendo un valor de 0.111. Esta predicción es un promedio de las 110 instancias de entrenamiento asociadas con este nodo hoja y tiene un ECM de 0.015.

¿Cómo particiona ahora un AD para regresión el espacio de los datos? Vamos a verlo en el siguiente ejemplo usando los datos anteriores:

In [ ]:
tree_reg1 = DecisionTreeRegressor(random_state=42, max_depth=2)
tree_reg2 = DecisionTreeRegressor(random_state=42, max_depth=3)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

def plot_regression_predictions(tree_reg, X, y, axes=[0, 1, -0.2, 1], ylabel="$y$"):
    x1 = np.linspace(axes[0], axes[1], 500).reshape(-1, 1)
    y_pred = tree_reg.predict(x1)
    plt.axis(axes)
    plt.xlabel("$x_1$", fontsize=18)
    if ylabel:
        plt.ylabel(ylabel, fontsize=18, rotation=0)
    plt.plot(X, y, "b.")
    plt.plot(x1, y_pred, "r.-", linewidth=2, label=r"$\hat{y}$")

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_regression_predictions(tree_reg1, X, y)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
plt.text(0.21, 0.65, "Depth=0", fontsize=15)
plt.text(0.01, 0.2, "Depth=1", fontsize=13)
plt.text(0.65, 0.8, "Depth=1", fontsize=13)
plt.legend(loc="upper center", fontsize=18)
plt.title("max_depth=2", fontsize=14)

plt.sca(axes[1])
plot_regression_predictions(tree_reg2, X, y, ylabel=None)
for split, style in ((0.1973, "k-"), (0.0917, "k--"), (0.7718, "k--")):
    plt.plot([split, split], [-0.2, 1], style, linewidth=2)
for split in (0.0458, 0.1298, 0.2873, 0.9040):
    plt.plot([split, split], [-0.2, 1], "k:", linewidth=1)
plt.text(0.3, 0.5, "Depth=2", fontsize=13)
plt.title("max_depth=3", fontsize=14)

plt.show()

Noten que el valor que predice siempre es un promedio de los datos que se encuentran en cada región. El algoritmo parte cada región de tal manera que elige a los puntos lo más cercanos posible para predecir el valor.

Al igual que el caso de clasificación, debemos tener cuidado ya que los AD pueden sobreajustarse a los datos. Por ejemplo, consideren dos modelos, uno sin restricciones, y otro con `min_samples_leaf = 10`.

In [ ]:
tree_reg1 = DecisionTreeRegressor(random_state=42)
tree_reg2 = DecisionTreeRegressor(random_state=42, min_samples_leaf=10)
tree_reg1.fit(X, y)
tree_reg2.fit(X, y)

x1 = np.linspace(0, 1, 500).reshape(-1, 1)
y_pred1 = tree_reg1.predict(x1)
y_pred2 = tree_reg2.predict(x1)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)

plt.sca(axes[0])
plt.plot(X, y, "b.")
plt.plot(x1, y_pred1, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", fontsize=18, rotation=0)
plt.legend(loc="upper center", fontsize=18)
plt.title("Sin restricciones", fontsize=14)

plt.sca(axes[1])
plt.plot(X, y, "b.")
plt.plot(x1, y_pred2, "r.-", linewidth=2, label=r"$\hat{y}$")
plt.axis([0, 1, -0.2, 1.1])
plt.xlabel("$x_1$", fontsize=18)
plt.title("min_samples_leaf={}".format(tree_reg2.min_samples_leaf), fontsize=14)

plt.show()

**Actividad**: ¿Qué pueden concluir de la gráfica anterior?

*Su respuesta en esta celda*

## Inestabilidad

Los AD son un ejemplo de un modelo de aprendizaje que cuenta con un gran balance entre interpretabilidad y versatilidad al momento de usarse. Pero, como todo en la vida, tiene sus limitaciones. Primero, los AD generan límites de decisión ortogonales, lo cual genera problemas si los datos rotan. Por ejemplo, la siguiente gráfica muestra un conjunto de datos linealmente separable que puede ser fácilmente separado por un AD. Sin embargo, si los mismos datos se rotan 45°, el margen de decisión se vuelve muy rebuscado.

In [ ]:
np.random.seed(6)
Xs = np.random.rand(100, 2) - 0.5
ys = (Xs[:, 0] > 0).astype(np.float32) * 2

angle = np.pi / 4
rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
Xsr = Xs.dot(rotation_matrix)

tree_clf_s = DecisionTreeClassifier(random_state=42)
tree_clf_s.fit(Xs, ys)
tree_clf_sr = DecisionTreeClassifier(random_state=42)
tree_clf_sr.fit(Xsr, ys)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(tree_clf_s, Xs, ys, axes=[-0.7, 0.7, -0.7, 0.7], iris=False)
plt.sca(axes[1])
plot_decision_boundary(tree_clf_sr, Xsr, ys, axes=[-0.7, 0.7, -0.7, 0.7], iris=False)
plt.ylabel("")

plt.show()

Otro defecto que tienen es que, en general, son muy sensibles a pequeñas variaciones de los datos con los que se entrenan. Por ejemplo, si aleatorizamos los datos, obtenemos un modelo distinto al que obtuvimos anteriormente:

In [ ]:
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf_tweaked = DecisionTreeClassifier(max_depth=2, random_state=40)
tree_clf_tweaked.fit(X, y)

plt.figure(figsize=(8, 4))
plot_decision_boundary(tree_clf_tweaked, X, y, legend=False)
plt.plot([0, 7.5], [0.8, 0.8], "k-", linewidth=2)
plt.plot([0, 7.5], [1.75, 1.75], "k--", linewidth=2)
plt.text(1.0, 0.9, "Depth=0", fontsize=15)
plt.text(1.0, 1.80, "Depth=1", fontsize=13)

plt.show()

## Ejercicio

La principal actividad de esta práctica es entrenar un modelo de clasificación usando unos datos de Kaggle. En primer lugar, entren a la [siguiente liga](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database) y descargen los datos para después subirlos al entorno de programación en Google Colab.

**Actividad 1** : Describir el conjunto de datos que acaban de descargar.

- ¿Sobre qué trata este conjunto de datos? ¿Que intenta predecir?
- ¿Cuántas características tiene cada dato? Escribir una breve descripción de cada una de ellas.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

Para empezar el flujo de trabajo, vamos a cargar los datos en un `DataFrame` de Pandas usando la función `read_csv`.

In [ ]:
df = pd.read_csv("diabetes.csv")

Nunca está de más verificar que todo se cargó sin problemas con el método `head()`.

In [ ]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Vamos a preparar los datos para el modelo partiendolos en un conjunto de prueba y uno de entrenamiento:

In [ ]:
feature_cols = ['pregnancies', 'glucose', 'bp', 'skinthickness','insulin','bmi','pedigree','age']
X = df.drop(['Outcome'], axis=1) # caracteristicas (todas salvo la columna 'outcome')
y = df['Outcome'] # variable objetivo ('outcome')

In [ ]:
# creamos los conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

Ahora vamos a crear nuestro modelo de clasificación, un AD para clasificación:

In [ ]:
# creamos el modelo de clasificacion
clf = DecisionTreeClassifier()

# entrenamos el modelo con nuestros datos de entrenamiento X_test y y_test
clf = clf.fit(X_train,y_train)

# realizamos las prediciones de X_test
y_pred = clf.predict(X_test)

Vamos a evaluar el rendimiento del modelo usando la medida F1

In [ ]:
# Model Accuracy, how often is the classifier correct?
print(f"F1 score: {f1_score(y_test, y_pred)}", )

F1 score: 0.5925925925925926


Como en los casos anteriores, vamos a visualizar el árbol y contemplar nuestro modelo entrenado:

In [ ]:
from sklearn.tree import export_graphviz
from six import StringIO
from IPython.display import Image
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('diabetes.png')
Image(graph.create_png())

¡Vaya! Está horrible. Bueno, es lo que salió. Problemas complejos requieren árboles complejos, ¿no creen? Bien, el único detalle que se omotió (por fines educativos) es la regularización del modelo.

**Actividad #2**: Vamos a medir el efecto que tiene el parámetro `max_depth` en la métrica de evaluación. Construir la gráfica de max_depth vs F1 score, con max_depth tomando valores desde 1 hasta 20. Para el modelo, utilizar como criterio `entropy`.

**Actividad #3**: Repetir la actividad anterior pero con otro parámetro que se discutió en la sección de regularización que no sea `max_depth`.